In [1]:
# CELL 1: Setup and Imports
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🐺 NOTEBOOK 6: PRE-GAME SIGNAL DETECTOR")
print("="*50)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("\nTesting Fenrir's Dance Hypotheses...")

/home/codespace/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🐺 NOTEBOOK 6: PRE-GAME SIGNAL DETECTOR
Analysis Date: 2026-01-06 23:29

Testing Fenrir's Dance Hypotheses...


In [2]:
# CELL 2: Define Universe (Same as Previous Notebooks)
UNIVERSE = {
    'Quantum': ['IONQ', 'RGTI', 'QBTS', 'QUBT'],
    'Nuclear': ['CCJ', 'UUUU', 'LEU', 'OKLO', 'SMR'],
    'Space': ['RKLB', 'LUNR', 'ASTS', 'RDW', 'SIDU'],
    'Semi': ['NVDA', 'MU', 'AEVA', 'LAZR', 'NVTS', 'SKYT'],
    'AI_Infra': ['VRT', 'SMCI', 'LITE'],
    'Rare_Earth': ['MP', 'USAR']
}

# Flatten for analysis
ALL_TICKERS = [t for tickers in UNIVERSE.values() for t in tickers]
print(f"Universe: {len(ALL_TICKERS)} tickers")
print(f"Sectors: {list(UNIVERSE.keys())}")

Universe: 25 tickers
Sectors: ['Quantum', 'Nuclear', 'Space', 'Semi', 'AI_Infra', 'Rare_Earth']


In [3]:
# CELL 3: Fetch Data with Extended Hours
# yfinance can show pre/post market data with interval='1h' or smaller

def fetch_extended_hours_data(ticker, period='6mo'):
    """Fetch data including after-hours and pre-market indicators"""
    try:
        stock = yf.Ticker(ticker)
        
        # Daily data for main analysis
        daily = stock.history(period=period, interval='1d')
        
        if daily.empty:
            return None
        
        # Calculate key metrics
        daily['Return'] = daily['Close'].pct_change() * 100
        daily['Avg_Volume_20d'] = daily['Volume'].rolling(20).mean()
        daily['Volume_Ratio'] = daily['Volume'] / daily['Avg_Volume_20d']
        
        # Gap calculation (Open vs Previous Close)
        daily['Prev_Close'] = daily['Close'].shift(1)
        daily['Gap_Pct'] = ((daily['Open'] - daily['Prev_Close']) / daily['Prev_Close']) * 100
        
        # Intraday range
        daily['Intraday_Range'] = ((daily['High'] - daily['Low']) / daily['Open']) * 100
        
        # Close position in range (0 = closed at low, 1 = closed at high)
        daily['Close_Position'] = (daily['Close'] - daily['Low']) / (daily['High'] - daily['Low'])
        
        return daily
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return None

# Fetch all data
print("Fetching extended data for all tickers...")
all_data = {}
for ticker in ALL_TICKERS:
    data = fetch_extended_hours_data(ticker)
    if data is not None and len(data) > 30:
        all_data[ticker] = data
        print(f"  ✓ {ticker}: {len(data)} days")
    else:
        print(f"  ✗ {ticker}: Insufficient data")

print(f"\nSuccessfully loaded: {len(all_data)} tickers")

Fetching extended data for all tickers...
  ✓ IONQ: 128 days
  ✓ RGTI: 128 days
  ✓ QBTS: 128 days
  ✓ QUBT: 128 days
  ✓ CCJ: 128 days
  ✓ UUUU: 128 days
  ✓ LEU: 128 days
  ✓ OKLO: 128 days
  ✓ SMR: 128 days
  ✓ RKLB: 128 days
  ✓ LUNR: 128 days
  ✓ ASTS: 128 days
  ✓ RDW: 128 days
  ✓ SIDU: 128 days
  ✓ NVDA: 128 days
  ✓ MU: 128 days
  ✓ AEVA: 128 days
  ✓ LAZR: 127 days
  ✓ NVTS: 128 days
  ✓ SKYT: 128 days
  ✓ VRT: 128 days
  ✓ SMCI: 128 days
  ✓ LITE: 128 days
  ✓ MP: 128 days
  ✓ USAR: 128 days

Successfully loaded: 25 tickers


In [4]:
# CELL 4: Identify All Runs (Same Logic as Notebook 1)

def find_runs(df, min_gain=10, min_days=2):
    """Find all upward runs in the data"""
    runs = []
    
    i = 0
    while i < len(df) - 1:
        # Look for start of potential run (green day)
        if df['Return'].iloc[i] > 0:
            start_idx = i
            start_price = df['Open'].iloc[i]
            
            # Extend run while mostly green
            j = i + 1
            red_count = 0
            while j < len(df) and red_count < 2:
                if df['Return'].iloc[j] < -1:  # Significant red day
                    red_count += 1
                else:
                    red_count = 0  # Reset on green/flat
                j += 1
            
            end_idx = j - 2 if red_count >= 2 else j - 1
            
            if end_idx > start_idx:
                end_price = df['Close'].iloc[end_idx]
                run_gain = ((end_price - start_price) / start_price) * 100
                run_days = end_idx - start_idx + 1
                
                if run_gain >= min_gain and run_days >= min_days:
                    runs.append({
                        'start_idx': start_idx,
                        'end_idx': end_idx,
                        'start_date': df.index[start_idx],
                        'end_date': df.index[end_idx],
                        'start_price': start_price,
                        'end_price': end_price,
                        'gain_pct': run_gain,
                        'days': run_days
                    })
            
            i = end_idx + 1
        else:
            i += 1
    
    return runs

# Find all runs for all tickers
all_runs = {}
total_runs = 0

for ticker, df in all_data.items():
    runs = find_runs(df)
    if runs:
        all_runs[ticker] = runs
        total_runs += len(runs)

print(f"\n🎯 Found {total_runs} total runs across {len(all_runs)} tickers")


🎯 Found 126 total runs across 25 tickers


---
## HYPOTHESIS 1: VOLUME DIVERGENCE BEFORE RUNS
### Does "Price Down + Volume Up" on Day -1 or Day 0 predict runs?

In [5]:
# CELL 5: Test Volume Divergence Hypothesis

def analyze_pre_run_signals(ticker, df, runs):
    """Analyze what happened on Day -2, Day -1, Day 0 before each run"""
    results = []
    
    for run in runs:
        day1_idx = run['start_idx']
        
        # Need at least 3 days of data before the run
        if day1_idx < 3:
            continue
        
        # Day 0 = day before run starts
        day0_idx = day1_idx - 1
        day_minus1_idx = day1_idx - 2
        day_minus2_idx = day1_idx - 3
        
        result = {
            'ticker': ticker,
            'run_start': run['start_date'],
            'run_gain': run['gain_pct'],
            'run_days': run['days'],
            
            # Day 0 (day before run)
            'day0_return': df['Return'].iloc[day0_idx],
            'day0_volume_ratio': df['Volume_Ratio'].iloc[day0_idx],
            'day0_gap': df['Gap_Pct'].iloc[day0_idx],
            'day0_close_position': df['Close_Position'].iloc[day0_idx],
            
            # Day -1 (2 days before run)
            'day_m1_return': df['Return'].iloc[day_minus1_idx],
            'day_m1_volume_ratio': df['Volume_Ratio'].iloc[day_minus1_idx],
            
            # Day -2 (3 days before run)
            'day_m2_return': df['Return'].iloc[day_minus2_idx],
            'day_m2_volume_ratio': df['Volume_Ratio'].iloc[day_minus2_idx],
            
            # Day 1 (first day of run)
            'day1_gap': df['Gap_Pct'].iloc[day1_idx],
            'day1_return': df['Return'].iloc[day1_idx],
            'day1_volume_ratio': df['Volume_Ratio'].iloc[day1_idx],
        }
        
        # Calculate signals
        result['day0_is_red'] = result['day0_return'] < 0
        result['day0_volume_elevated'] = result['day0_volume_ratio'] > 1.5
        result['volume_divergence'] = result['day0_is_red'] and result['day0_volume_elevated']
        result['day0_closed_high'] = result['day0_close_position'] > 0.7  # Closed near high despite red
        result['accumulation_signal'] = result['day0_is_red'] and result['day0_closed_high']
        
        # Multi-day red setup
        result['red_days_before'] = sum([
            1 if result['day0_return'] < 0 else 0,
            1 if result['day_m1_return'] < 0 else 0,
            1 if result['day_m2_return'] < 0 else 0
        ])
        
        results.append(result)
    
    return results

# Analyze all runs
all_pre_run_signals = []

for ticker, runs in all_runs.items():
    df = all_data[ticker]
    signals = analyze_pre_run_signals(ticker, df, runs)
    all_pre_run_signals.extend(signals)

signals_df = pd.DataFrame(all_pre_run_signals)
print(f"\nAnalyzed {len(signals_df)} run setups")
print(f"\n" + "="*60)


Analyzed 116 run setups



In [6]:
# CELL 6: HYPOTHESIS 1 RESULTS - Volume Divergence

print("\n🔬 HYPOTHESIS 1: VOLUME DIVERGENCE")
print("Does 'Price Down + Volume Up' on Day 0 predict runs?")
print("="*60)

# Filter valid data
valid = signals_df.dropna(subset=['day0_volume_ratio', 'day0_return'])

# Calculate stats
total = len(valid)
day0_red = valid['day0_is_red'].sum()
day0_vol_up = valid['day0_volume_elevated'].sum()
vol_divergence = valid['volume_divergence'].sum()

print(f"\nTotal runs analyzed: {total}")
print(f"\nDay 0 Characteristics:")
print(f"  Day 0 is RED: {day0_red}/{total} ({day0_red/total*100:.1f}%)")
print(f"  Day 0 Volume > 1.5x: {day0_vol_up}/{total} ({day0_vol_up/total*100:.1f}%)")
print(f"  VOLUME DIVERGENCE (Red + High Vol): {vol_divergence}/{total} ({vol_divergence/total*100:.1f}%)")

# Compare gains: Volume Divergence vs No Volume Divergence
with_vd = valid[valid['volume_divergence'] == True]
without_vd = valid[valid['volume_divergence'] == False]

print(f"\n📊 DOES VOLUME DIVERGENCE PREDICT BIGGER RUNS?")
print(f"\n  WITH Volume Divergence ({len(with_vd)} runs):")
if len(with_vd) > 0:
    print(f"    Avg Run Gain: +{with_vd['run_gain'].mean():.1f}%")
    print(f"    Avg Run Days: {with_vd['run_days'].mean():.1f}")
    print(f"    Avg Day 1 Gap: +{with_vd['day1_gap'].mean():.1f}%")

print(f"\n  WITHOUT Volume Divergence ({len(without_vd)} runs):")
if len(without_vd) > 0:
    print(f"    Avg Run Gain: +{without_vd['run_gain'].mean():.1f}%")
    print(f"    Avg Run Days: {without_vd['run_days'].mean():.1f}")
    print(f"    Avg Day 1 Gap: +{without_vd['day1_gap'].mean():.1f}%")

# Verdict
if len(with_vd) > 5 and len(without_vd) > 5:
    gain_diff = with_vd['run_gain'].mean() - without_vd['run_gain'].mean()
    print(f"\n🎯 VERDICT: Volume Divergence adds {gain_diff:+.1f}% to avg run gain")
    if gain_diff > 5:
        print("   ✅ VALIDATED - Volume divergence IS a useful signal!")
    elif gain_diff > 0:
        print("   ⚠️ WEAK - Slight edge but not significant")
    else:
        print("   ❌ KILLED - Volume divergence does NOT predict better runs")


🔬 HYPOTHESIS 1: VOLUME DIVERGENCE
Does 'Price Down + Volume Up' on Day 0 predict runs?

Total runs analyzed: 105

Day 0 Characteristics:
  Day 0 is RED: 105/105 (100.0%)
  Day 0 Volume > 1.5x: 8/105 (7.6%)
  VOLUME DIVERGENCE (Red + High Vol): 8/105 (7.6%)

📊 DOES VOLUME DIVERGENCE PREDICT BIGGER RUNS?

  WITH Volume Divergence (8 runs):
    Avg Run Gain: +48.5%
    Avg Run Days: 14.2
    Avg Day 1 Gap: +2.4%

  WITHOUT Volume Divergence (97 runs):
    Avg Run Gain: +38.6%
    Avg Run Days: 12.1
    Avg Day 1 Gap: +0.5%

🎯 VERDICT: Volume Divergence adds +9.9% to avg run gain
   ✅ VALIDATED - Volume divergence IS a useful signal!


---
## HYPOTHESIS 2: ACCUMULATION SIGNAL
### Does "Red Day + Close Near High" indicate smart money accumulation?

In [7]:
# CELL 7: HYPOTHESIS 2 - Accumulation Signal

print("\n🔬 HYPOTHESIS 2: ACCUMULATION SIGNAL")
print("Does 'Red Day + Close Near High' predict bigger runs?")
print("="*60)

valid = signals_df.dropna(subset=['day0_close_position', 'day0_return'])

# Accumulation = Red day but closed in upper 30% of range
accum_signal = valid['accumulation_signal'].sum()

print(f"\nTotal runs: {len(valid)}")
print(f"Day 0 Accumulation Signal (Red + Close High): {accum_signal}/{len(valid)} ({accum_signal/len(valid)*100:.1f}%)")

# Compare
with_accum = valid[valid['accumulation_signal'] == True]
without_accum = valid[valid['accumulation_signal'] == False]

print(f"\n📊 DOES ACCUMULATION SIGNAL PREDICT BIGGER RUNS?")
print(f"\n  WITH Accumulation Signal ({len(with_accum)} runs):")
if len(with_accum) > 0:
    print(f"    Avg Run Gain: +{with_accum['run_gain'].mean():.1f}%")
    print(f"    Avg Day 1 Gain: +{with_accum['day1_return'].mean():.1f}%")
    print(f"    Avg Day 1 Gap: +{with_accum['day1_gap'].mean():.1f}%")

print(f"\n  WITHOUT Accumulation Signal ({len(without_accum)} runs):")
if len(without_accum) > 0:
    print(f"    Avg Run Gain: +{without_accum['run_gain'].mean():.1f}%")
    print(f"    Avg Day 1 Gain: +{without_accum['day1_return'].mean():.1f}%")
    print(f"    Avg Day 1 Gap: +{without_accum['day1_gap'].mean():.1f}%")

if len(with_accum) > 5 and len(without_accum) > 5:
    gain_diff = with_accum['run_gain'].mean() - without_accum['run_gain'].mean()
    print(f"\n🎯 VERDICT: Accumulation signal adds {gain_diff:+.1f}% to avg run gain")
    if gain_diff > 5:
        print("   ✅ VALIDATED - Accumulation signal IS useful!")
    elif gain_diff > 0:
        print("   ⚠️ WEAK - Slight edge but not significant")
    else:
        print("   ❌ KILLED - Accumulation signal does NOT predict better runs")


🔬 HYPOTHESIS 2: ACCUMULATION SIGNAL
Does 'Red Day + Close Near High' predict bigger runs?

Total runs: 116
Day 0 Accumulation Signal (Red + Close High): 7/116 (6.0%)

📊 DOES ACCUMULATION SIGNAL PREDICT BIGGER RUNS?

  WITH Accumulation Signal (7 runs):
    Avg Run Gain: +48.7%
    Avg Day 1 Gain: +6.0%
    Avg Day 1 Gap: +1.7%

  WITHOUT Accumulation Signal (109 runs):
    Avg Run Gain: +37.6%
    Avg Day 1 Gain: +4.6%
    Avg Day 1 Gap: +0.7%

🎯 VERDICT: Accumulation signal adds +11.1% to avg run gain
   ✅ VALIDATED - Accumulation signal IS useful!


---
## HYPOTHESIS 3: MULTI-DAY RED SETUP
### Do 2-3 red days before a run predict BIGGER explosions?

In [8]:
# CELL 8: HYPOTHESIS 3 - Multi-Day Red Setup

print("\n🔬 HYPOTHESIS 3: MULTI-DAY RED SETUP")
print("Do more red days before a run predict bigger gains?")
print("="*60)

valid = signals_df.dropna(subset=['red_days_before'])

# Group by number of red days
print(f"\n📊 RUN PERFORMANCE BY RED DAYS BEFORE:")
print(f"\n| Red Days Before | Count | Avg Run Gain | Avg Day 1 Gap | Avg Run Days |")
print(f"|-----------------|-------|--------------|---------------|--------------|")

results_by_red = []
for red_days in [0, 1, 2, 3]:
    subset = valid[valid['red_days_before'] == red_days]
    if len(subset) > 0:
        results_by_red.append({
            'red_days': red_days,
            'count': len(subset),
            'avg_gain': subset['run_gain'].mean(),
            'avg_day1_gap': subset['day1_gap'].mean(),
            'avg_days': subset['run_days'].mean()
        })
        print(f"| {red_days}               | {len(subset):5} | +{subset['run_gain'].mean():11.1f}% | +{subset['day1_gap'].mean():12.1f}% | {subset['run_days'].mean():12.1f} |")

# Analysis
if len(results_by_red) >= 2:
    gains = [r['avg_gain'] for r in results_by_red]
    red_days = [r['red_days'] for r in results_by_red]
    
    # Trend analysis
    if len(gains) >= 3:
        correlation = np.corrcoef(red_days, gains)[0, 1]
        print(f"\n🎯 CORRELATION (Red Days vs Gain): {correlation:.2f}")
        
        if correlation > 0.3:
            print("   ✅ VALIDATED - More red days = bigger runs!")
        elif correlation > 0:
            print("   ⚠️ WEAK - Slight positive trend")
        else:
            print("   ❌ KILLED - Red days don't predict run size")


🔬 HYPOTHESIS 3: MULTI-DAY RED SETUP
Do more red days before a run predict bigger gains?

📊 RUN PERFORMANCE BY RED DAYS BEFORE:

| Red Days Before | Count | Avg Run Gain | Avg Day 1 Gap | Avg Run Days |
|-----------------|-------|--------------|---------------|--------------|
| 2               |    46 | +       38.8% | +         1.0% |         12.8 |
| 3               |    70 | +       37.9% | +         0.6% |         11.4 |


---
## HYPOTHESIS 4: DAY 1 GAP PREDICTION
### Can we predict Day 1 gap size from Day 0 signals?

In [9]:
# CELL 9: HYPOTHESIS 4 - Gap Prediction

print("\n🔬 HYPOTHESIS 4: DAY 1 GAP PREDICTION")
print("Can Day 0 signals predict if Day 1 will gap up big?")
print("="*60)

valid = signals_df.dropna(subset=['day1_gap', 'day0_volume_ratio'])

# Define "big gap" as > 3%
valid['big_gap'] = valid['day1_gap'] > 3

# What predicts big gaps?
big_gaps = valid[valid['big_gap'] == True]
small_gaps = valid[valid['big_gap'] == False]

print(f"\nTotal runs: {len(valid)}")
print(f"Big gaps (>3%): {len(big_gaps)} ({len(big_gaps)/len(valid)*100:.1f}%)")
print(f"Small gaps (≤3%): {len(small_gaps)} ({len(small_gaps)/len(valid)*100:.1f}%)")

print(f"\n📊 DAY 0 CHARACTERISTICS BEFORE BIG vs SMALL GAPS:")
print(f"\n| Metric | Big Gap (>3%) | Small Gap (≤3%) |")
print(f"|--------|---------------|-----------------|")

metrics = [
    ('Avg Day 0 Return', 'day0_return'),
    ('Avg Day 0 Volume Ratio', 'day0_volume_ratio'),
    ('% Day 0 Red', 'day0_is_red'),
    ('% Volume Divergence', 'volume_divergence'),
    ('% Accumulation Signal', 'accumulation_signal'),
    ('Avg Red Days Before', 'red_days_before'),
]

for name, col in metrics:
    if col in ['day0_is_red', 'volume_divergence', 'accumulation_signal']:
        big_val = big_gaps[col].mean() * 100
        small_val = small_gaps[col].mean() * 100
        print(f"| {name:25} | {big_val:12.1f}% | {small_val:14.1f}% |")
    else:
        big_val = big_gaps[col].mean()
        small_val = small_gaps[col].mean()
        print(f"| {name:25} | {big_val:13.2f} | {small_val:15.2f} |")

# Key finding
print(f"\n🎯 KEY FINDING:")
vd_big = big_gaps['volume_divergence'].mean() * 100
vd_small = small_gaps['volume_divergence'].mean() * 100
print(f"   Volume Divergence before big gaps: {vd_big:.1f}%")
print(f"   Volume Divergence before small gaps: {vd_small:.1f}%")
if vd_big > vd_small * 1.5:
    print("   ✅ Volume divergence DOES predict bigger gaps!")
else:
    print("   ❌ Volume divergence does NOT predict gap size")


🔬 HYPOTHESIS 4: DAY 1 GAP PREDICTION
Can Day 0 signals predict if Day 1 will gap up big?

Total runs: 105
Big gaps (>3%): 15 (14.3%)
Small gaps (≤3%): 90 (85.7%)

📊 DAY 0 CHARACTERISTICS BEFORE BIG vs SMALL GAPS:

| Metric | Big Gap (>3%) | Small Gap (≤3%) |
|--------|---------------|-----------------|
| Avg Day 0 Return          |         -4.95 |           -4.83 |
| Avg Day 0 Volume Ratio    |          1.02 |            0.94 |
| % Day 0 Red               |        100.0% |          100.0% |
| % Volume Divergence       |         13.3% |            6.7% |
| % Accumulation Signal     |          6.7% |            6.7% |
| Avg Red Days Before       |          2.67 |            2.62 |

🎯 KEY FINDING:
   Volume Divergence before big gaps: 13.3%
   Volume Divergence before small gaps: 6.7%
   ✅ Volume divergence DOES predict bigger gaps!


---
## HYPOTHESIS 5: THE COMBINED PRE-GAME SIGNAL
### Can we combine signals to find Day 0 setups?

In [10]:
# CELL 10: Combined Signal Score

print("\n🔬 HYPOTHESIS 5: COMBINED PRE-GAME SIGNAL")
print("Can we stack signals to predict Day 0 better?")
print("="*60)

valid = signals_df.dropna()

# Create a "Pre-Game Score" (0-5 points)
def calculate_pregame_score(row):
    score = 0
    
    # +1: Day 0 is red
    if row['day0_is_red']:
        score += 1
    
    # +1: Volume elevated on Day 0
    if row['day0_volume_ratio'] > 1.5:
        score += 1
    
    # +1: Volume divergence (red + high vol)
    if row['volume_divergence']:
        score += 1
    
    # +1: Accumulation signal (red + close high)
    if row['accumulation_signal']:
        score += 1
    
    # +1: Multiple red days before (2+)
    if row['red_days_before'] >= 2:
        score += 1
    
    return score

valid['pregame_score'] = valid.apply(calculate_pregame_score, axis=1)

print(f"\n📊 RUN PERFORMANCE BY PRE-GAME SCORE:")
print(f"\n| Score | Count | Avg Run Gain | Avg Day 1 Gap | Avg Day 1 Return |")
print(f"|-------|-------|--------------|---------------|------------------|")

for score in sorted(valid['pregame_score'].unique()):
    subset = valid[valid['pregame_score'] == score]
    print(f"| {score}     | {len(subset):5} | +{subset['run_gain'].mean():11.1f}% | +{subset['day1_gap'].mean():12.1f}% | +{subset['day1_return'].mean():15.1f}% |")

# High score vs low score
high_score = valid[valid['pregame_score'] >= 3]
low_score = valid[valid['pregame_score'] <= 1]

print(f"\n🎯 HIGH SCORE (3+) vs LOW SCORE (0-1):")
print(f"\n  HIGH Score ({len(high_score)} runs):")
if len(high_score) > 0:
    print(f"    Avg Run Gain: +{high_score['run_gain'].mean():.1f}%")
    print(f"    Avg Day 1 Gap: +{high_score['day1_gap'].mean():.1f}%")
    print(f"    Avg Day 1 Return: +{high_score['day1_return'].mean():.1f}%")

print(f"\n  LOW Score ({len(low_score)} runs):")
if len(low_score) > 0:
    print(f"    Avg Run Gain: +{low_score['run_gain'].mean():.1f}%")
    print(f"    Avg Day 1 Gap: +{low_score['day1_gap'].mean():.1f}%")
    print(f"    Avg Day 1 Return: +{low_score['day1_return'].mean():.1f}%")

if len(high_score) > 3 and len(low_score) > 3:
    gain_diff = high_score['run_gain'].mean() - low_score['run_gain'].mean()
    gap_diff = high_score['day1_gap'].mean() - low_score['day1_gap'].mean()
    print(f"\n🎯 VERDICT:")
    print(f"   High pre-game score adds {gain_diff:+.1f}% to run gain")
    print(f"   High pre-game score adds {gap_diff:+.1f}% to Day 1 gap")


🔬 HYPOTHESIS 5: COMBINED PRE-GAME SIGNAL
Can we stack signals to predict Day 0 better?

📊 RUN PERFORMANCE BY PRE-GAME SCORE:

| Score | Count | Avg Run Gain | Avg Day 1 Gap | Avg Day 1 Return |
|-------|-------|--------------|---------------|------------------|
| 2     |    88 | +       37.9% | +         0.4% | +            4.5% |
| 3     |     6 | +       51.8% | +         1.9% | +            5.4% |
| 4     |     8 | +       48.5% | +         2.4% | +            4.8% |

🎯 HIGH SCORE (3+) vs LOW SCORE (0-1):

  HIGH Score (14 runs):
    Avg Run Gain: +49.9%
    Avg Day 1 Gap: +2.2%
    Avg Day 1 Return: +5.0%

  LOW Score (0 runs):


---
## 🎯 CURRENT DAY 0 CANDIDATES
### Which stocks show pre-game signals RIGHT NOW?

In [11]:
# CELL 11: Current Day 0 Scanner

print("\n🔍 CURRENT DAY 0 CANDIDATES")
print("Scanning for pre-game signals as of today...")
print("="*60)

candidates = []

for ticker, df in all_data.items():
    if len(df) < 5:
        continue
    
    # Get last few days
    recent = df.tail(5)
    today_idx = -1
    
    # Today's stats
    today_return = df['Return'].iloc[today_idx]
    today_vol_ratio = df['Volume_Ratio'].iloc[today_idx]
    today_close_pos = df['Close_Position'].iloc[today_idx]
    
    # Recent returns
    recent_returns = df['Return'].iloc[-3:].tolist()
    red_days = sum(1 for r in recent_returns if r < 0)
    
    # Calculate score
    score = 0
    signals = []
    
    if today_return < 0:
        score += 1
        signals.append("RED today")
    
    if today_vol_ratio > 1.5:
        score += 1
        signals.append(f"HIGH VOL ({today_vol_ratio:.1f}x)")
    
    if today_return < 0 and today_vol_ratio > 1.5:
        score += 1
        signals.append("VOL DIVERGENCE")
    
    if today_return < 0 and today_close_pos > 0.7:
        score += 1
        signals.append("ACCUMULATION")
    
    if red_days >= 2:
        score += 1
        signals.append(f"{red_days} RED DAYS")
    
    candidates.append({
        'ticker': ticker,
        'score': score,
        'today_return': today_return,
        'vol_ratio': today_vol_ratio,
        'close_position': today_close_pos,
        'red_days': red_days,
        'signals': signals
    })

# Sort by score
candidates = sorted(candidates, key=lambda x: x['score'], reverse=True)

print(f"\n🚨 TOP DAY 0 CANDIDATES (Score 2+):")
print(f"\n| Ticker | Score | Today | Vol Ratio | Signals |")
print(f"|--------|-------|-------|-----------|---------|")

for c in candidates:
    if c['score'] >= 2:
        signals_str = ', '.join(c['signals'][:3])
        print(f"| {c['ticker']:6} | {c['score']}     | {c['today_return']:+5.1f}% | {c['vol_ratio']:8.1f}x | {signals_str} |")

print(f"\n⚠️ These stocks may be in Day 0 setup for a run!")
print(f"Watch for gap up tomorrow to confirm Day 1.")


🔍 CURRENT DAY 0 CANDIDATES
Scanning for pre-game signals as of today...

🚨 TOP DAY 0 CANDIDATES (Score 2+):

| Ticker | Score | Today | Vol Ratio | Signals |
|--------|-------|-------|-----------|---------|
| UUUU   | 3     |  -0.7% |      1.5x | RED today, HIGH VOL (1.5x), VOL DIVERGENCE |
| NVDA   | 2     |  -0.5% |      1.1x | RED today, 2 RED DAYS |
| LAZR   | 2     |  -2.3% |      0.0x | RED today, 2 RED DAYS |

⚠️ These stocks may be in Day 0 setup for a run!
Watch for gap up tomorrow to confirm Day 1.


---
## 📊 FINAL SUMMARY: HYPOTHESIS RESULTS

In [12]:
# CELL 12: Final Summary

print("\n" + "="*70)
print("🐺 NOTEBOOK 6: FINAL HYPOTHESIS RESULTS")
print("="*70)

print("""
┌─────────────────────────────────────────────────────────────────────┐
│                    PRE-GAME SIGNAL TEST RESULTS                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  HYPOTHESIS 1: Volume Divergence (Red + High Vol)                   │
│  → Does elevated volume on red days predict runs?                   │
│                                                                     │
│  HYPOTHESIS 2: Accumulation Signal (Red + Close High)               │
│  → Does closing high on red days indicate smart money?              │
│                                                                     │
│  HYPOTHESIS 3: Multi-Day Red Setup                                  │
│  → Do more red days before = bigger runs?                           │
│                                                                     │
│  HYPOTHESIS 4: Gap Prediction                                       │
│  → Can Day 0 signals predict Day 1 gap size?                        │
│                                                                     │
│  HYPOTHESIS 5: Combined Pre-Game Score                              │
│  → Does stacking signals improve prediction?                        │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
""")

print("\n📋 ACTIONABLE TAKEAWAYS:")
print("")
print("1. Check the score breakdown above - higher score = better Day 0 setup")
print("2. Look at current Day 0 candidates - these may run tomorrow")
print("3. Compare with Brokkr's existing signals for confirmation")
print("")
print("🐺 THE HUNT CONTINUES...")


🐺 NOTEBOOK 6: FINAL HYPOTHESIS RESULTS

┌─────────────────────────────────────────────────────────────────────┐
│                    PRE-GAME SIGNAL TEST RESULTS                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  HYPOTHESIS 1: Volume Divergence (Red + High Vol)                   │
│  → Does elevated volume on red days predict runs?                   │
│                                                                     │
│  HYPOTHESIS 2: Accumulation Signal (Red + Close High)               │
│  → Does closing high on red days indicate smart money?              │
│                                                                     │
│  HYPOTHESIS 3: Multi-Day Red Setup                                  │
│  → Do more red days before = bigger runs?                           │
│                                                                     │
│  HYPOTHESIS 4: Gap Pr